In [ ]:
# Display the 2 images
from IPython.display import Image, display

img_paths = ['/home/up2121537/kaggle/steel_detection/test/000ccc2ac.jpg',
            '/home/up2121537/kaggle/steel_detection/test/0000f269f.jpg']
display(Image(img_paths [0]))
display(Image(img_paths [1]))

In [ ]:
# Read the CSV file
lables = pd.read_csv("/home/up2121537/kaggle/steel_detection/traincsv/train.csv")

In [ ]:
# Show the CSV file
lables

In [ ]:
# Visualize the distribution of the class labels in a dataset
lables["ClassId"].value_counts().plot(kind = 'bar')
lables["ClassId"].value_counts()

In [ ]:
# Organize the images into classes
from __future__ import print_function
import pandas as pd
import shutil 
import os 
import sys

lables = pd.read_csv("/home/up2121537/kaggle/steel_detection/traincsv/train.csv")

train_dir = '/home/up2121537/kaggle/steel_detection/train'
Class = '/home/up2121537/kaggle/steel_detection/Class'

if not os.path.exists(Class):
    os.mkdir(Class)
    
for row in lables.itertuples(): 
    ImageId = row.ImageId
    ClassId = row.ClassId
    
    if not os.path.exists(Class + str(ClassId)):
        os.mkdir(Class + str(ClassId))
        
    src_path = train_dir + '/' + ImageId 
    dst_path = Class + str(ClassId) + '/' + ImageId 
    
    try: 
        shutil.copy2(src_path, dst_path)
        print("successful")
    except IOError as e: 
        print ('Unable to copy file {} to {}'
               .format(src_path, dst_path))
    except:
        print('When try copy file {} to {}, unexpected error: {}'
              .format(src_path, dst_path, sys.exc_info()))

In [ ]:
# Import the necessary libraries
import os        
import numpy as np # linear algebra
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import PIL
import PIL.Image
from tensorflow import keras
import matplotlib.pyplot as plt


In [ ]:
# Model 1 using a pretrainde model(Resnet 50) with 10 epochs

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

# Define the model architecture
base_model = ResNet50(input_shape=(120, 120, 3), include_top=False, weights='imagenet')
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['acc'])

# Create an instance of ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Define the data directories
data_dir = '/home/up2121537/kaggle/steel_detection/data/Class'

# Flow the data from the directories and apply data augmentation
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(120, 120),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(120, 120),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Train the model
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//validation_generator.batch_size,
    epochs=10,
    verbose=1
)

# Plot the training and validation accuracy and loss
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Evaluate the model on the testing data
test_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(120, 120),
    batch_size=32,
    class_mode='categorical'
)

test_loss, test_acc = model.evaluate(test_generator, verbose=1)
print('Test accuracy:', test_acc)

In [ ]:
# model 2 just 3 CNN layers , the size of kernel is (3,3) with 10 epoches and using data augmentation
 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(120, 120, 3)),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['acc'])

# Create an instance of ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Define the data directories
data_dir = '/home/up2121537/kaggle/steel_detection/data/Class'

# Flow the data from the directories and apply data augmentation
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(120, 120),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(120, 120),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//validation_generator.batch_size,
    epochs=10,
    verbose=1
)

# Plot the training and validation accuracy and loss
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(10)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Evaluate the model on the testing data
test_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(120, 120),
    batch_size=32,
    class_mode='categorical'
)
test_loss, test_acc = model.evaluate_generator(test_generator, steps=test_generator.samples//test_generator.batch_size)

print('Test Acc:', test_acc)

In [ ]:
# model 3
# the most accurate model with 3 CNN layers with maxpooling and Dropout , the size of kernel is (5,5) with 50 epoches and using data augmentation

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt


# Define the model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), strides=(2, 2) ,activation='relu', input_shape=(120, 120, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    keras.layers.Dropout(rate=0.15),
    tf.keras.layers.Conv2D(32, (3,3), strides=(2, 2) ,activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    keras.layers.Dropout(rate=0.1),
    tf.keras.layers.Conv2D(64, (3,3), strides=(2, 2),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    keras.layers.Dropout(rate=0.10),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['acc'])

# Create an instance of ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
# Define the data directories
data_dir = '/home/up2121537/kaggle/steel_detection/data/Class'

# Flow the data from the directories and apply data augmentation
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(120, 120),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(120, 120),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples//validation_generator.batch_size,
    epochs=50,
    verbose=1
)

# Plot the training and validation accuracy and loss
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(50)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Evaluate the model on the testing data
test_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(120, 120),
    batch_size=32,
    class_mode='categorical'
)
test_loss, test_acc = model.evaluate_generator(test_generator, steps=test_generator.samples//test_generator.batch_size)

print('Test Acc:', test_acc)
